[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_oil_stats/EMQA_oil_stats.ipynb)

# EMQA_oil_stats

Brent crude oil key statistics.

**Output:** `oil_statistics.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
from scipy import stats as sp_stats

brent = fetch('BZ=F')
ret = np.log(brent / brent.shift(1)).dropna()

print("=== Brent Crude Oil Statistics (2020-2025) ===")
print(f"Mean Price:    ${brent.mean():.2f}")
print(f"Median Price:  ${brent.median():.2f}")
print(f"Std Dev:       ${brent.std():.2f}")
print(f"Min:           ${brent.min():.2f}")
print(f"Max:           ${brent.max():.2f}")
print(f"Mean Return:   {ret.mean()*100:.3f}%")
print(f"Return Vol:    {ret.std()*100:.2f}%")
print(f"Skewness:      {sp_stats.skew(ret):.2f}")
print(f"Kurtosis:      {sp_stats.kurtosis(ret, fisher=False):.2f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))
ax1.hist(brent, bins=40, color=COLORS['blue'], alpha=0.7, edgecolor='white')
ax1.axvline(brent.mean(), color=COLORS['red'], ls='--', label=f'Mean ${brent.mean():.0f}')
ax1.set_title('Price Distribution', fontweight='bold')
ax1.set_xlabel('USD/bbl')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), frameon=False)

ax2.hist(ret*100, bins=60, color=COLORS['red'], alpha=0.7, edgecolor='white', density=True)
x = np.linspace(ret.min()*100, ret.max()*100, 200)
ax2.plot(x, sp_stats.norm.pdf(x, ret.mean()*100, ret.std()*100), color=COLORS['blue'], lw=2, label='Normal fit')
ax2.set_title('Return Distribution (Fat Tails)', fontweight='bold')
ax2.set_xlabel('Daily Return (%)')
ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), frameon=False)

fig.tight_layout()
save_fig(fig, 'oil_statistics.pdf')
plt.show()
